Optimizing ML to reach 75% or higher

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
optimized_app_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
optimized_app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
optimized_app_df = optimized_app_df.drop(columns=['EIN'])
optimized_app_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Determine the number of unique values in each column.
optimized_app_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [9]:
# Look at NAME value counts for binning
name = optimized_app_df.NAME.value_counts()
name

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
optimized_app_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Choosing a cut off value for names with values <10 and creat a list of names to be replaced by 'others'

name_replace = list(name[name<10].index)
# Replace in dataframe using a for loop
for name in name_replace:
  optimized_app_df['NAME'] = optimized_app_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
optimized_app_df['NAME'].value_counts()

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
optimized_app_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# You may find it helpful to look at CLASSIFICATION value counts >1
clss_cnts = optimized_app_df['CLASSIFICATION'].value_counts()
clssifcation_to_replace = clss_cnts.loc[clss_cnts > 1]
clssifcation_to_replace

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = clss_cnts.loc[clss_cnts < 1000].index
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    optimized_app_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
optimized_app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
opt_app_dummies_df = pd.get_dummies(optimized_app_df)
opt_app_dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
# Split our preprocessed data into our features and target arrays
y1 = opt_app_dummies_df['IS_SUCCESSFUL']
X1 = opt_app_dummies_df.drop(columns=['IS_SUCCESSFUL'], inplace=False)

# Split the preprocessed data into a training and testing dataset
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=.5, random_state=1)

In [20]:
# Create a StandardScaler instances
scaler1 = StandardScaler()

# Fit the StandardScaler
X1_scaler = scaler1.fit(X1_train)

# Scale the data
X1_train_scaled = X1_scaler.transform(X1_train)
X1_test_scaled = X1_scaler.transform(X1_test)

Compile, Train and Evaluate the new mode

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=274))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                22000     
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 24,461
Trainable params: 24,461
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model1 = nn1.fit(X1_train_scaled, y1_train, epochs=25)

Epoch 1/25
536/536 [==============================] - 3s 3ms/step - loss: 0.4887 - accuracy: 0.7653
Epoch 2/25
536/536 [==============================] - 1s 2ms/step - loss: 0.4459 - accuracy: 0.7833
Epoch 3/25
536/536 [==============================] - 1s 2ms/step - loss: 0.4406 - accuracy: 0.7853
Epoch 4/25
536/536 [==============================] - 1s 2ms/step - loss: 0.4380 - accuracy: 0.7871
Epoch 5/25
536/536 [==============================] - 1s 2ms/step - loss: 0.4367 - accuracy: 0.7892
Epoch 6/25
536/536 [==============================] - 1s 2ms/step - loss: 0.4348 - accuracy: 0.7887
Epoch 7/25
536/536 [==============================] - 1s 2ms/step - loss: 0.4339 - accuracy: 0.7897
Epoch 8/25
536/536 [==============================] - 1s 2ms/step - loss: 0.4327 - accuracy: 0.7912
Epoch 9/25
536/536 [==============================] - 2s 3ms/step - loss: 0.4322 - accuracy: 0.7897
Epoch 10/25
536/536 [==============================] - 2s 3ms/step - loss: 0.4316 - accuracy: 0.7900

In [25]:
# Evaluate the model using the test data
model_loss1, model_accuracy1 = nn1.evaluate(X1_test_scaled,y1_test,verbose=2)
print(f"Loss: {model_loss1}, Accuracy: {model_accuracy1}")

536/536 - 1s - loss: 0.4511 - accuracy: 0.7806 - 1s/epoch - 2ms/step
Loss: 0.45109546184539795, Accuracy: 0.7805830836296082


In [26]:
# Export our model to HDF5 file
nn1.save('AlphabetSoupCharityOptimized_model.h5')

Additional attempts that did not garner more than 72% accuracy

In [ ]:
# optomize attempt 2 the model by adjusting layers

# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# nn_2 = tf.keras.models.Sequential()

# # First hidden layer
# nn_2.add(tf.keras.layers.Dense(units=375, activation="relu", input_dim=2))

# # Second hidden layer
# nn_2.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# # Output layer
# nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
# nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 375)               1125      
                                                                 
 dense_6 (Dense)             (None, 30)                11280     
                                                                 
 dense_7 (Dense)             (None, 1)                 31        
                                                                 
Total params: 12,436
Trainable params: 12,436
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# # Compile the model
# nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# # Train the model
# fit_model_2 = nn_2.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
536/536 [==============================] - 2s 2ms/step - loss: 0.5706 - accuracy: 0.7210
Epoch 2/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7298
Epoch 3/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7311
Epoch 4/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5491 - accuracy: 0.7335
Epoch 5/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7332
Epoch 6/25
536/536 [==============================] - 1s 3ms/step - loss: 0.5467 - accuracy: 0.7350
Epoch 7/25
536/536 [==============================] - 1s 3ms/step - loss: 0.5461 - accuracy: 0.7344
Epoch 8/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7343
Epoch 9/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5448 - accuracy: 0.7352
Epoch 10/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5440 - accuracy: 0.7361

In [ ]:
# # Evaluate the model using the test data
# model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

536/536 - 1s - loss: 0.5558 - accuracy: 0.7281 - 669ms/epoch - 1ms/step
Loss: 0.5557909607887268, Accuracy: 0.7281049489974976


Third attempt to optimize

In [ ]:
# # Create a method that creates a new Sequential model with hyperparameter options
# def create_model(hp):
#     nn_model = tf.keras.models.Sequential()

#     # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
#     # Allow kerastuner to decide number of neurons in first layer
#     nn_model.add(tf.keras.Dense(units=hp.Int('first_units',
#         min_value=1,
#         max_value=10,
#         step=2), activation=activation, input_dim=2))

#     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#     for i in range(hp.Int('num_layers', 1, 6)):
#         nn_model.add(tf.keras.Dense(units=hp.Int('units_' + str(i),
#             min_value=1,
#             max_value=10,
#             step=2),
#             activation=activation))
    
#     nn_model.add(tf.keras.Dense(units=1, activation="sigmoid"))

#     # Compile the model
#     nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
#     return nn_model

In [ ]:
# # Import the kerastuner library
# import keras_tuner as kt

# tuner = kt.Hyperband(
#     create_model,
#     objective="val_accuracy",
#     max_epochs=20,
#     hyperband_iterations=2)

In [ ]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))


Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
sigmoid           |relu              |activation
3                 |7                 |first_units
2                 |3                 |num_layers
7                 |9                 |units_0
5                 |5                 |units_1
7                 |None              |units_2
3                 |None              |units_3
9                 |None              |units_4
7                 |None              |units_5
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/tuners/hyperband.py", line 425, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 213, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 155, in _try_build
    model = self._bui

RuntimeError: ignored

In [ ]:
# optomize attempt 3 the model by adjusting layers

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# nn_3 = tf.keras.models.Sequential()

# # First hidden layer
# nn_3.add(tf.keras.layers.Dense(units=9, activation="relu", input_dim=43))

# # Second hidden layer
# nn_3.add(tf.keras.layers.Dense(units=5, activation="sigmoid"))

# # Output layer
# nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
# nn_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 9)                 396       
                                                                 
 dense_7 (Dense)             (None, 5)                 50        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 452
Trainable params: 452
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
# nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
# fit_model_3 = nn_3.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
536/536 [==============================] - 2s 2ms/step - loss: 0.6341 - accuracy: 0.6621
Epoch 2/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5740 - accuracy: 0.7276
Epoch 3/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5662 - accuracy: 0.7299
Epoch 4/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5622 - accuracy: 0.7301
Epoch 5/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5593 - accuracy: 0.7307
Epoch 6/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5567 - accuracy: 0.7305
Epoch 7/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5550 - accuracy: 0.7305
Epoch 8/25
536/536 [==============================] - 1s 3ms/step - loss: 0.5540 - accuracy: 0.7316
Epoch 9/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5530 - accuracy: 0.7309
Epoch 10/25
536/536 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7326

In [ ]:
# Evaluate the model using the test data
# model_loss_3, model_accuracy_3 = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

536/536 - 1s - loss: 0.5568 - accuracy: 0.7258 - 650ms/epoch - 1ms/step
Loss: 0.5567769408226013, Accuracy: 0.7257726192474365
